### 1.0 Importando Bibliotecas

In [20]:
# Abrir a conexão com o banco de dados
import sqlite3
import pandas as pd
conn = sqlite3.connect('database.db')

### 2.0 Coletando Dados

#### 2.1 Juntando as tabelas

In [21]:
consulta_atividade = """ 
  SELECT
    *
  FROM flight_activity fa LEFT JOIN flight_loyalty_history flh ON (fa. loyalty_number = flh.loyalty_number)
"""

df_atividade = pd.read_sql_query (consulta_atividade, conn)
df_atividade.head(5)

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


### 3.0 Inspecionando as Tabelas

In [22]:
df_atividade.shape

(405624, 26)

In [23]:
df_atividade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loyalty_number               405624 non-null  int64  
 1   year                         405624 non-null  int64  
 2   month                        405624 non-null  int64  
 3   flights_booked               405624 non-null  int64  
 4   flights_with_companions      405624 non-null  int64  
 5   total_flights                405624 non-null  int64  
 6   distance                     405624 non-null  int64  
 7   points_accumulated           405624 non-null  float64
 8   points_redeemed              405624 non-null  int64  
 9   dollar_cost_points_redeemed  405624 non-null  int64  
 10  loyalty_number               405624 non-null  int64  
 11  country                      405624 non-null  object 
 12  province                     405624 non-null  object 
 13 

### 4.0 Exercícios SQL

### 5.0 Preparação dos dados

In [24]:
# Número de dados faltantes em cada coluna
df_atividade.isna().sum()

# Selecionar as colunas que somente contém numeros
colunas = ["year", "month", "flights_booked", "flights_with_companions", "total_flights", "distance", "points_accumulated", "salary", "clv", "loyalty_card"]

df_colunas_numericas = df_atividade.loc[:, colunas]


# Remover as linhas que contem dados faltantes
df_dados_completos = df_colunas_numericas.dropna()


# Verificar se existem dados faltantes
df_dados_completos.isna().sum()

year                       0
month                      0
flights_booked             0
flights_with_companions    0
total_flights              0
distance                   0
points_accumulated         0
salary                     0
clv                        0
loyalty_card               0
dtype: int64

In [25]:
df_dados_completos.shape[0]

302952

### 6.0 Machine Learning

In [26]:
df_dados_completos.head()

,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,salary,clv,loyalty_card
0,2017,1,3,0,3,1521,152.0,92552.0,7919.20,Aurora
3,2017,1,0,0,0,0,0.0,63253.0,4170.57,Star
4,2017,1,0,0,0,0,0.0,91163.0,6622.05,Star
5,2017,1,0,0,0,0,0.0,70323.0,48356.96,Nova
6,2017,1,0,0,0,0,0.0,76849.0,5143.88,Nova


In [27]:
from sklearn import tree as tr

X_atributos = df_dados_completos.drop(columns = 'loyalty_card')
y_rotulos = df_dados_completos.loc[:, 'loyalty_card']

# Definição do algoritmo
modelo = tr.DecisionTreeClassifier(max_depth = 3 )

# Treinamento do algoritmo
modelo_treinado = modelo.fit( X_atributos , y_rotulos)

In [31]:

tree = tr.plot_tree(modelo_treinado , filled=True)
tree

[Text(0.375, 0.875, 'x[8] <= 2739.87\ngini = 0.641\nsamples = 302952\nvalue = [64512, 105288, 133152]'),
 Text(0.25, 0.625, 'gini = 0.0\nsamples = 32232\nvalue = [0, 0, 32232]'),
 Text(0.5, 0.625, 'x[8] <= 4936.925\ngini = 0.653\nsamples = 270720\nvalue = [64512, 105288, 100920]'),
 Text(0.25, 0.375, 'x[8] <= 3842.41\ngini = 0.5\nsamples = 74040\nvalue = [0, 37944, 36096]'),
 Text(0.125, 0.125, 'gini = 0.44\nsamples = 36720\nvalue = [0.0, 24720.0, 12000.0]'),
 Text(0.375, 0.125, 'gini = 0.458\nsamples = 37320\nvalue = [0, 13224, 24096]'),
 Text(0.75, 0.375, 'x[8] <= 40436.32\ngini = 0.667\nsamples = 196680\nvalue = [64512, 67344, 64824]'),
 Text(0.625, 0.125, 'gini = 0.667\nsamples = 194616\nvalue = [63576, 66600, 64440]'),
 Text(0.875, 0.125, 'gini = 0.63\nsamples = 2064\nvalue = [936, 744, 384]')]

### 7.0 Apresentando o Resultado

In [29]:
x_novo = X_atributos.sample()
previsao = modelo_treinado.predict_proba (x_novo)

print('Prob - Aurora: {:.1f}% - Nova: {:.1f}% - Star: {:.1f}%'.format(100*previsao[0][0], 100*previsao[0][1], 100*previsao[0][2]))

Prob - Aurora: 32.7% - Nova: 34.2% - Star: 33.1%


### 8.0 Painel de Visualização

In [10]:
import gradio as gr
import numpy as np

In [11]:
X_atributos.loc[:, 'year'].max()

2018

In [12]:
def predict(*args):
    x_novo = np.array([args]).reshape(1, -1)
    previsao = modelo_treinado.predict_proba (x_novo)
    
    return {"Aurora":previsao[0][0], "Nova":previsao[0][1], "Star":previsao[0][2]}

with gr.Blocks() as demo:
    # Titulo do Painel
    gr.Markdown (""" # Propensão de Compra """)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown (""" Atributos do Cliente """)
            year = gr.Slider(label ='Ano', minimum=2017, maximum=2018, step=1, randomize=True)
            month = gr.Slider(label ='Mês', minimum=1, maximum=12, step=1, randomize=True)
            flights_booked = gr.Slider(label ='Voos Reservados', minimum=0, maximum=21, step=1, randomize=True)
            flights_with_companions = gr.Slider(label ='Voos com Companhias', minimum=0, maximum=11, step=1, randomize=True)
            total_flights = gr.Slider(label ='Total de Voos', minimum=0, maximum=32, step=1, randomize=True)
            distance = gr.Slider(label ='Distância Percorrida no Voo', minimum=0, maximum=6293, step=1, randomize=True)
            points_accumulated = gr.Slider(label ='Pontos Acumulados', minimum=0.00, maximum=676.50, step=1, randomize=True)
            salary = gr.Slider(label ='Renda Anual', minimum=58486.00, maximum=407228.00, step=1, randomize=True)
            clv = gr.Slider(label ='Valor Total para Todos os Voos', minimum=2119.89, maximum=83325.38, step=1, randomize=True)
            
            with gr.Row():
                gr.Markdown (""" Botão de Previsão """)
                predict_btn = gr.Button(value='Previsão')
            
        with gr.Column():
            gr.Markdown (""" Propensão de Compra do Cliente """)
            label = gr.Label()
    
    # Botão predict
    predict_btn.click(
        fn=predict,
        inputs = [
            year,
            month,
            flights_booked,
            flights_with_companions,
            total_flights,
            distance,
            points_accumulated,
            salary,
            clv
            ],
        outputs=[label])
    
    
    
demo.launch(debug=True, share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\marce\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\marce\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\marce\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\marce\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\marce\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid

Keyboard interruption in main thread... closing server.
